# Extending Python with C Manually

Such extension modules can do two things that can’t be done directly in Python:
- they can implement new built-in object types.
- they can call C library functions and system calls.

The Python API is incorporated in a C source file by including the header `"Python.h"`.

## Create a C Module
### Writing C header and source file 

In [1]:
%%file fact.h
#ifndef FACT_H
#define FACT_h

int fact(int n);

#endif

Overwriting fact.h


In [2]:
%%file fact.c
#include "fact.h"
    
int fact(int n) {
    if (n <= 1) {
        return 1;
    }
    else {
        return n * fact(n - 1);
    }
}

Overwriting fact.c


### Writing C wrap function

In [3]:
%%file fact_wrap.c

/* Must include Python.h before any standard headers*/
#include <Python.h>
#include "fact.h"
static PyObject* wrap_fact(PyObject *self, PyObject *args)
{
    /* Python->C data conversion */
    int n, result;
    // the string i here means there is only one integer
    if (!PyArg_ParseTuple(args, "i", &n))
        return NULL;
    
    /* C Function Call */
    result = fact(n);
    
    /* C->Python data conversion */
    return Py_BuildValue("i", result);
}

/* Method table declaring the names of functions exposed to Python*/
static PyMethodDef ExampleMethods[] = {
    {"fact",  wrap_fact, METH_VARARGS, "Calculate the factorial of n"},
    {NULL, NULL, 0, NULL}        /* Sentinel */
};

/* Module initialization function called at "import example"*/
PyMODINIT_FUNC 
initexample(void)
{
    (void) Py_InitModule("example", ExampleMethods);
}

Overwriting fact_wrap.c


## Compile the C  Module
### Compile with gcc manually
### Compile with setup.py

In [4]:
%%file setup.py
from distutils.core import setup, Extension

ext = Extension(name='example', sources=['fact_wrap.c', 'fact.c'])

setup(name='example', ext_modules=[ext])

Overwriting setup.py


In [5]:
!python setup.py build_ext --inplace

running build_ext
building 'example' extension
gcc -pthread -B /home/liyj/anaconda3/compiler_compat -Wl,--sysroot=/ -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -fPIC -I/home/liyj/anaconda3/include/python3.6m -c fact_wrap.c -o build/temp.linux-x86_64-3.6/fact_wrap.o
fact_wrap.c: In function ‘initexample’:
fact_wrap.c:30:12: warning: implicit declaration of function ‘Py_InitModule’ [-Wimplicit-function-declaration]
     (void) Py_InitModule("example", ExampleMethods);
            ^
fact_wrap.c:31:1: warning: control reaches end of non-void function [-Wreturn-type]
 }
 ^
gcc -pthread -B /home/liyj/anaconda3/compiler_compat -Wl,--sysroot=/ -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -fPIC -I/home/liyj/anaconda3/include/python3.6m -c fact.c -o build/temp.linux-x86_64-3.6/fact.o
gcc -pthread -shared -B /home/liyj/anaconda3/compiler_compat -L/home/liyj/anaconda3/lib -Wl,-rpath=/home/liyj/anaconda3/lib -Wl,--no-as-needed -Wl,--sysroot=/ build/temp.linu

### Using compiled module

In [6]:
import example
dir(example)

ImportError: /home/liyj/git/learn-python3/08-extending-and-embedding/example.cpython-36m-x86_64-linux-gnu.so: undefined symbol: Py_InitModule